# Data Acquisition and Deployment

## Model Deployment

In [ ]:
import tensorflow as tf
from IPython.display import Image, display

import matplotlib.pyplot as plt
import tensorflow_hub as hub

import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [ ]:
!nvidia-smi

## Open Source DB

In [ ]:
!wget https://storage.googleapis.com/ibeans/train.zip
!wget https://storage.googleapis.com/ibeans/validation.zip
!wget https://storage.googleapis.com/ibeans/test.zip



!unzip train.zip -d beans/
!unzip validation.zip -d beans/
!unzip test.zip -d beans/

!ls beans

!find beans -type f | wc -l
!find beans/test -type f | wc -l
!find beans/validation -type f | wc -l

## Implicit Data Structure

Use internal folder structure as classes

In [ ]:
!mkdir beans

In [ ]:
!unzip train.zip -d beans/
!unzip validation.zip -d beans/
!unzip test.zip -d beans/

In [ ]:
!ls beans

In [ ]:
!find beans/train -type f | wc -l

In [ ]:
!find beans/test -type f | wc -l

In [ ]:
!find beans/validation -type f | wc -l

In [ ]:
display(Image('beans/train/healthy/healthy_train.265.jpg'))

In [ ]:
display(Image('beans/train/angular_leaf_spot/angular_leaf_spot_train.98.jpg'))

In [ ]:
display(Image('beans/train/bean_rust/bean_rust_train.212.jpg'))

## Parameterization

In [ ]:
batch_size = 128
img_height = 224
img_width = 224

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory('beans/train',
            seed=111,
            image_size=(img_width, img_height),
            batch_size=batch_size)

In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory('beans/test',
            seed=111,
            image_size=(img_width, img_height),
            batch_size=batch_size)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory('beans/validation',
            seed=111,
            image_size=(img_width, img_height),
            batch_size=batch_size)

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
classes=train_ds.class_names
print(classes)

In [ ]:
plt.figure(figsize=(10, 10))

for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype('uint8'))
    plt.title(classes[labels[i]])
    plt.axis('off')

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Model Acquisition and Deployment

## tf_hub model import

https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4

In [ ]:
feature_extractor = 'https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4'

In [ ]:
feature_extractor_layer = hub.KerasLayer(feature_extractor, input_shape=(img_height,img_width,3))

In [ ]:
feature_extractor_layer.trainable = False

In [ ]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [ ]:
tf.random.set_seed(111)

In [ ]:
model = tf.keras.Sequential([
                             normalization_layer,
                             feature_extractor_layer,
                             tf.keras.layers.Dropout(0.3),
                             tf.keras.layers.Dense(3,activation='softmax')
])

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

## Train & Tests model

In [ ]:
history = model.fit(train_ds, epochs=20, validation_data=val_ds)

In [ ]:
model.summary()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train_acc', 'val_acc'], loc='best')
plt.show()

In [ ]:
result = model.evaluate(test_ds)

In [ ]:
plt.figure(figsize=(10, 10))

for images, labels in test_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.tight_layout()

    img = tf.keras.preprocessing.image.img_to_array(images[i])
    img = np.expand_dims(img, axis=0)

    pred = model.predict(img)
    plt.imshow(images[i].numpy().astype('uint8'))
    plt.title('Real Label: %s' % classes[labels[i]])
    plt.text(1, 240, 'Predicted Label: %s' % classes[np.argmax(pred)], fontsize=12)

    plt.axis('off')

## Save model

In [ ]:
model.save('./models', save_format='tf')

In [ ]:
# !ls models
!ls -alrt models


## Inference on a single image

In [ ]:
model_loaded = tf.keras.models.load_model('./models')

In [ ]:
model_loaded.summary()

## Trasform inference image to fit model

In [ ]:
from PIL import Image
import numpy as np
from skimage import transform

In [ ]:
def process_img(filename):
  np_image = Image.open(filename)
  np_image = np.array(np_image).astype('float32')
  np_image = transform.resize(np_image, (224, 224, 3))
  np_image = np.expand_dims(np_image, axis=0)
  return np_image

## Final Test

In [ ]:
pred_label = model_loaded.predict(process_img('beans/train/healthy/healthy_train.265.jpg'))
print(classes[np.argmax(pred_label)])

In [ ]:
pred_label

# Export Model to Deploy

In [ ]:
print('TensorFlow version: ' + tf.__version__)

In [ ]:
!zip -r models.zip models/